In [13]:
# set parameters
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import dmg5e
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [14]:
# import data
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD.apply(lambda row: estats5e.effHP(row['HP Mean'], row['AC']), axis=1)
dfD['eDPR Mean'] = dfD.apply(lambda row: estats5e.effDPR(row['DPR Mean'], row['AB']), axis=1)
dfD['eXP Mean']  = dfD.apply(lambda row: estats5e.effXP(row['HP Mean'], row['AC'], row['DPR Mean'], row['AB']), axis=1)
dfD.set_index('CR', inplace=True)

df0 = pd.read_csv('../../assets/data/monsters.csv')  
# 'Book','Page','Type','Category','Legendary','CR','PB','XP',
# 'HP','AC','adj HP','adj AC',
# 'AB','adj AB','DPR', 'adj DPR'

df0 = df0[df0['DPR'].gt(0) & ~df0['DPR'].isna()]
df0['D-CR DMG'] = df0.apply(lambda row: dmg5e.monster_defensive_challenge_rating(row['adj HP'], row['adj AC']), axis=1)
df0['O-CR DMG'] = df0.apply(lambda row: dmg5e.monster_offensive_challenge_rating(row['adj DPR'], row['adj AB']), axis=1)
df0['C-CR DMG'] = df0.apply(lambda row: np.floor(dmg5e.monster_challenge_rating(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB'])), axis=1)

df0['D-CR DMG Delta'] = df0['D-CR DMG'] - df0['CR']
df0['O-CR DMG Delta'] = df0['O-CR DMG'] - df0['CR']
df0['C-CR DMG Delta'] = df0['C-CR DMG'] - df0['CR']

df0['D-CR DMG unadj'] = df0.apply(lambda row: dmg5e.monster_defensive_challenge_rating(row['HP'], row['AC']), axis=1)
df0['O-CR DMG unadj'] = df0.apply(lambda row: dmg5e.monster_offensive_challenge_rating(row['DPR'], row['AB']), axis=1)
df0['C-CR DMG unadj'] = df0.apply(lambda row: np.floor(dmg5e.monster_challenge_rating(row['HP'], row['AC'], row['DPR'], row['AB'])), axis=1)

print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1792 monsters in database


In [15]:
# Setup template and html config for plotly figures.
def plot_confidence_interval(fig, df0, conf, xCol, yCol, name, line_color, fillcolor):
    # plot confidence
    if conf > 0:
        dfU = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
        dfL = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()
        fig.add_trace(go.Scatter(x=dfU[xCol], y=dfU[yCol], line_width=0, legendgroup=name, showlegend=False, hoverinfo='skip'))
        fig.add_trace(go.Scatter(x=dfL[xCol], y=dfL[yCol], line_width=0, name=f'{name} - {conf:.0%} confidence', legendgroup=name, hoverinfo='skip', fill='tonexty', fillcolor=fillcolor))
    
    # plot mean
    dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='lines', 
        name=f'{name} - mean',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.1f}<br>' + '<extra></extra>'
    ))

In [16]:
# plots monster HP vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit points',
            range=[0,1000],
            tick0=0, dtick=200,
            minor=dict(tick0=0, dtick=50),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'HP', 'HP', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj HP', 'adj HP', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['HP Mean'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-hp-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-hp-vs-cr')

In [17]:
# plots monster AC vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[10,28],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'AC', 'AC', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj AC', 'adj AC', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['AC'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-ac-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-ac-vs-cr-small')

In [18]:
# defensive CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='defensive CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'D-CR DMG', 'D-CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
    showlegend=False,
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-d-cr-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-d-cr-vs-cr')

In [19]:
# plots monster DPR vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0,320],
            tick0=0, dtick=50,
            minor=dict(tick0=0, dtick=25),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'DPR', 'DPR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj DPR', 'adj DPR', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['DPR Mean'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-dpr-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-dpr-vs-cr')

In [20]:
# plots monster AB vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[0,20],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'AB', 'AB', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj AB', 'adj AB', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['AB'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-ab-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-ab-vs-cr')

In [21]:
# offensive CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='offensive CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'O-CR DMG', 'O-CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
    showlegend=False,
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-o-cr-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-o-cr-vs-cr')

In [22]:
# final CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG', 'DMG CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-dmg-cr-vs-cr')
    tfb.save_fig_html(fig, format='small', name='fig-dmg-cr-vs-cr')

In [23]:
# final CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG unadj', 'unadjusted CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG', 'adjusted CR', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip', 
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-adj-cr-vs-cr')
    #tfb.save_fig_html(fig, format='small', name='fig-adj-cr-vs-cr')
    pass